In [1]:
import os
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

## IMDB 데이터세트 다운로드

In [2]:
# tensorflow_datasets에서 imdb의 영화리뷰 데이터를 가져온다. 
# as_supervised의 기본값은 False다 차이는 True는 튜플로 False는 딕셔너리로 데이터를 가져온다.
train_data, validation_data, test_data = tfds.load(
    name = 'imdb_reviews',
    split = ('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True
)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteG28GFI/imdb_reviews-train.tfrecord…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteG28GFI/imdb_reviews-test.tfrecord*…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteG28GFI/imdb_reviews-unsupervised.t…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [4]:
# 가져온 테스트 데이터를 확인해본다.
train_example_batch, train_labels_batch = next(iter(train_data.batch(3)))
train_example_batch, train_labels_batch

(<tf.Tensor: shape=(3,), dtype=string, numpy=
 array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
        b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fel

# TF-Hub에서 임베딩 모델을 가져온다.

In [5]:
# TF-hub에서 사전 학습된 임베딩 모델을 가져온다. 
embedding = 'https://tfhub.dev/google/nnlm-en-dim50/2'
hub_layer = hub.KerasLayer(embedding, input_shape=[],
                           dtype=tf.string, trainable = True)
hub_layer(train_example_batch[:2])

<tf.Tensor: shape=(2, 50), dtype=float32, numpy=
array([[ 0.5423195 , -0.0119017 ,  0.06337538,  0.06862972, -0.16776837,
        -0.10581174,  0.16865303, -0.04998824, -0.31148055,  0.07910346,
         0.15442263,  0.01488662,  0.03930153,  0.19772711, -0.12215476,
        -0.04120981, -0.2704109 , -0.21922152,  0.26517662, -0.80739075,
         0.25833532, -0.3100421 ,  0.28683215,  0.1943387 , -0.29036492,
         0.03862849, -0.7844411 , -0.0479324 ,  0.4110299 , -0.36388892,
        -0.58034706,  0.30269456,  0.3630897 , -0.15227164, -0.44391504,
         0.19462997,  0.19528408,  0.05666234,  0.2890704 , -0.28468323,
        -0.00531206,  0.0571938 , -0.3201318 , -0.04418665, -0.08550783,
        -0.55847436, -0.23336391, -0.20782952, -0.03543064, -0.17533456],
       [ 0.56338924, -0.12339553, -0.10862679,  0.7753425 , -0.07667089,
        -0.15752277,  0.01872335, -0.08169781, -0.3521876 ,  0.4637341 ,
        -0.08492756,  0.07166859, -0.00670817,  0.12686075, -0.19326553,
 

In [6]:
# tf-hub의 임베딩 레이어를 추가한 간단한 모델을 구성해 본다.
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation ='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 50)                48190600  
                                                                 
 dense (Dense)               (None, 16)                816       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 48,191,433
Trainable params: 48,191,433
Non-trainable params: 0
_________________________________________________________________


In [7]:
# loss_function과 optimizer를 설정해준다.
model.compile(optimizer='adam',
              loss= tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics = ['accuracy'])

In [9]:
# 모델을 훈련한다.
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs = 10,
                    validation_data = validation_data.batch(512),
                    verbose=1)

Epoch 1/10
30/30 [==============================] - 9s 288ms/step - loss: 0.6668 - accuracy: 0.5611 - val_loss: 0.6089 - val_accuracy: 0.6041
Epoch 2/10
30/30 [==============================] - 8s 283ms/step - loss: 0.5420 - accuracy: 0.6967 - val_loss: 0.5070 - val_accuracy: 0.7410
Epoch 3/10
30/30 [==============================] - 8s 279ms/step - loss: 0.4138 - accuracy: 0.8161 - val_loss: 0.4117 - val_accuracy: 0.8203
Epoch 4/10
30/30 [==============================] - 9s 288ms/step - loss: 0.3026 - accuracy: 0.8853 - val_loss: 0.3515 - val_accuracy: 0.8377
Epoch 5/10
30/30 [==============================] - 8s 282ms/step - loss: 0.2230 - accuracy: 0.9222 - val_loss: 0.3220 - val_accuracy: 0.8580
Epoch 6/10
30/30 [==============================] - 9s 288ms/step - loss: 0.1660 - accuracy: 0.9467 - val_loss: 0.3092 - val_accuracy: 0.8674
Epoch 7/10
30/30 [==============================] - 9s 287ms/step - loss: 0.1223 - accuracy: 0.9655 - val_loss: 0.3063 - val_accuracy: 0.8719
Epoch 

In [11]:
# 테스트 데이터를 통해 해당 모델의 훈련이 잘 되었는지 확인해 본다.
model.evaluate(test_data.batch(512),verbose=1)

49/49 [==============================] - 4s 71ms/step - loss: 0.3563 - accuracy: 0.8504


[0.3562571108341217, 0.8504400253295898]